In [1]:
import h5py    
import numpy as np  
import bluepysnap
import matplotlib.pyplot as plt
import seaborn as sns
import bluepysnap
from bluepysnap import Simulation as snap_sim
from pathlib import Path
# Create a custom legend
import matplotlib.patches as mpatches

plt.rc('text', usetex=True)

In [2]:
# Add path for the config folder where circuit_config of the simulation is
results_path = '/gpfs/bbp.cscs.ch/data/project/proj137/farinaNGV/Paper_results/config/'

In [3]:
# read the circuit from ../../config/circuit_config.json
# NOTE: in circuit_config.json many path are relative
# We need to read the circuit to get the info about the neurons as e-type and layer
#circuit = bluepysnap.Circuit('/gpfs/bbp.cscs.ch/data/project/proj137/farinaNGV/Paper_results/config/circuit_config.json')
circuit = bluepysnap.Circuit('../../config/circuit_config.json' )
node_population = circuit.nodes["All"]
pandas_nodes = node_population.get()

In [4]:
def read_result(result_path):
    read_result = h5py.File(result_path)
    result = read_result['report']['All']['data']

    return(result)

def mapping_extract(result_path):
    read_result = h5py.File(result_path)
    result = read_result['report']['All']['data']
        
    extract_mapping = read_result['report']['All']['mapping']['node_ids'][:]
    
    t0 = read_result['report']['All']['mapping']['time'][0]
    T =read_result['report']['All']['mapping']['time'][1]
    dt = read_result['report']['All']['mapping']['time'][2]

    
    N= (T-t0)/dt
    time_line = np.linspace(t0,T, int(N))
    N2 = (T-t0)/0.025
    voltage_time_line = np.linspace(t0,T,int(N2))
    return(extract_mapping, time_line, voltage_time_line)

In [5]:
# Path: Results Folder (where reporting_metabolism_young is) can be downloaded from 10.5281/zenodo.14187063
results_path='/gpfs/bbp.cscs.ch/data/project/proj137/farinaNGV/Paper_results/my_simulation/reporting_metabolism_young/'
read_nai = read_result(results_path + 'ndam_nai.h5')
mapping, time_line, volt_time = mapping_extract(results_path + 'ndam_nai.h5')

In [6]:
# Importing Pandas to create DataFrame
import pandas as pd
 
# Creating Empty DataFrame and Storing it in variable df
df = pd.DataFrame()

In [7]:
df['ids'] = mapping
df['etype'] = [pandas_nodes['etype'][i] for i in df['ids']]
df['layer'] = [pandas_nodes['layer'][i] for i in df['ids']]
df['synapse_class'] = [ pandas_nodes['synapse_class'][i] for i in df['ids']]

In [8]:
number_neurons = len(mapping)
number_astro = 85

In [9]:
count_exc = (df['synapse_class'] == 'EXC').sum()
count_inh = (df['synapse_class'] == 'INH').sum()
print(count_exc, count_inh)

24368 3594


In [10]:
dict_nodes_2 ={}
dict_nodes_2["Layer1"]=[]
dict_nodes_2["Layer2Excitatory"] =[]
dict_nodes_2["Layer2Inhibitory"] =[]
dict_nodes_2["Layer3Excitatory"] =[]
dict_nodes_2["Layer3Inhibitory"] =[]
dict_nodes_2["Layer4Excitatory"] =[]
dict_nodes_2["Layer4Inhibitory"] =[]
dict_nodes_2["Layer5Excitatory"] =[]
dict_nodes_2["Layer5Inhibitory"] =[]
dict_nodes_2["Layer6Excitatory"] =[]
dict_nodes_2["Layer6Inhibitory"] =[]

#node_population.get().index.to_list()
for j in mapping:
    cell_id = node_population.get(j)
    layer = cell_id.get('layer')
    syn_class = cell_id.get('synapse_class')
    if layer == 1:
        dict_nodes_2["Layer1"].append(j)
    elif syn_class == 'INH':
        if layer == 2:
            dict_nodes_2["Layer2Inhibitory"].append(j)
        elif layer == 3:
            dict_nodes_2["Layer3Inhibitory"].append(j)
        elif layer ==4:
            dict_nodes_2["Layer4Inhibitory"].append(j)
        elif layer==5:
            dict_nodes_2["Layer5Inhibitory"].append(j)
        elif layer==6:
            dict_nodes_2["Layer6Inhibitory"].append(j)
        else:
            print("not in the options!")
    elif syn_class == 'EXC':
        if layer == 2 :
            dict_nodes_2["Layer2Excitatory"].append(j)
        elif layer == 3 :
            dict_nodes_2["Layer3Excitatory"].append(j)
        elif layer ==4:
            dict_nodes_2["Layer4Excitatory" ].append(j)
        elif layer==5:
            dict_nodes_2["Layer5Excitatory"].append(j)
        elif layer==6:
            dict_nodes_2["Layer6Excitatory"].append(j)
        else:
            print("not in the options!")

In [11]:
Total_cells_subgroup = len(mapping)
print(Total_cells_subgroup)
def func2(key_call):
    values = len(dict_nodes_2[key_call]) *100./ Total_cells_subgroup
    return(values)
data_2 = [func2(j) for j in dict_nodes_2.keys()]
print('data_2', data_2)

27962
data_2 [1.326800658035906, 12.938988627422932, 2.667906444460339, 16.915814319433515, 1.7380730992060653, 20.538588083828053, 1.4162077104642015, 24.629854802946856, 4.101995565410199, 12.123596309276875, 1.6021743795150563]


In [14]:
# Save to a CSV file
#df.to_csv('/gpfs/bbp.cscs.ch/data/project/proj137/farinaNGV/Multiscale_ndm_met/Figures/Data/fig1_circuit_composition.csv', index=False)